<img src="./graphics/RUS_logo.jpg">

<center><h1>  Monitoring tropospheric NO2 with Sentinel-5P products using the Atmospheric Toolbox - Part 1      </h1></center>



***
**General Note 1**: Execute each cell through the <button class="btn btn-default btn-xs"><i class="icon-play fa fa-play"></i></button> button from the top MENU (or keyboard shortcut `Shift` + `Enter`).<br>
<br>
**General Note 2**: If, for any reason, the kernel is not working anymore, in the top MENU, click on the <button class="btn btn-default btn-xs"><i class="fa fa-repeat icon-repeat"></i></button> button. Then, in the top MENU, click on "Cell" and select "Run All Above Selected Cell".<br>
***

# Table of contents

- [1. Introduction](#1.-Introduction)
- [2. Installing and importing python libraries](#2.-Installing-and-importing-python-libraries)
- [3. Download Sentinel-5P data](#3.-Download-Sentinel-5P-data)
- [4. Explore the netCDF file content](#4.-Explore-the-netCDF-file-content)
  - [4.1. Product Exploration](#4.1.-Product-Exploration)
  - [4.2. Visualisation of one NO2 product](#4.2.-Visualisation-of-one-NO2-product)
- [5. Visualisation of NO2 for a day](#5.-Visualisation-of-NO2-for-a-day)
  - [5.1. Concatenation](#5.1.-Concatenation)
     - [5.1.1. Harpmerge in command line](#5.1.1.-Harpmerge-in-command-line)
     - [5.1.2. Harp.import_product() without preprocessing](#5.1.2.-Harp.import_product()-without-preprocessing)
     - [5.1.3. Harp.import_product() with preprocessing](#5.1.3.-Harp.import_product()-with-preprocessing)
  - [5.2. Visualisation of NO2](#5.2.-Visualisation-of-NO2)
     - [5.2.1. Visualisation of the three outputs](#5.2.1.-Visualisation-of-the-three-outputs)
     - [5.2.2. Visualisation of NO2 over Europe](#5.2.2.-Visualisation-of-NO2-over-Europe)
- [6. Conclusion](#6.-Conclusion)

# 1. Introduction
 [Go back to the "Table of contents"](#Table-of-contents)

<p style='text-align: justify;'> 
Air quality is one of the major threats to human health, accounting for 13% of the deaths in the European Union. Monitoring the air we breathe is key to developing mitigation strategies and quantify the effects of these policies. Pollutants are emitted from different sources, most of them being anthropogenic sources such as motor vehicles and industrial combustion processes. To implement new policies, authorities rely on space based technologies as they offer a unique way to monitor air pollutants daily and globally.

<p style='text-align: justify;'> 
In this context, Sentinel-5P was launched in October 2017. Its main purpose is to screen the Earth’s atmosphere and quantify different pollutants (CO, NO<sub>2</sub>, SO<sub>2</sub>, O<sub>3</sub>, aerosols…) with a great accuracy and spatial resolution. It also provides measurement continuity with precedent and ongoing atmospheric spatial missions (OMI, IASI and SCHIAMACHY). The data recorded by this satellite are free of use and present a great interest to globally monitor air quality, greenhouse gas emissions and detect and assess the impact of polluting events.</p>

<div class="alert alert-block alert-info">
<b>Tip:</b> More information on the characteristics of Sentinel-5p can be found <a href="https://sentinel.esa.int/web/sentinel/missions/sentinel-5p"> here </a>
</div>

<p style='text-align: justify;'> 
    This part of the training will focus on opening and exploring Sentinel-5P NO2 products with python. We will eventually apply some basic processing on the products to monitor NO2 in the troposphere (lower part of the atmosphere). The specifically designed <a href="https://atmospherictoolbox.org/"> Atmospheric Toolbox </a> and the <a href="http://xarray.pydata.org/en/stable/"> xarray </a> Python library will be used throughout the workshop. You will learn to monitor NO2 pollution based on processed Sentinel-5P data.

<p style='text-align: justify;'> 
    In this first part, we are going to introduce you to ingesting, reading and navigating through Sentinel-5P NO2 products with Python. This will include: 
    
* Downloading automatically S5P products by command line;
* Opening and exploring the downloaded products;
* Mapping the NO2 tropospheric column in one product;
* Three different ways of concatenating NO2 products to create averaged maps;
* Defining a specific study area and adapting the processing;
    
<p style='text-align: justify;'>
    The exercises will be implemented using Python code that can be found in this Jupyter Notebook. Although it is recommended to have some basic knowledge, the training does not require any Python programming skills and can be followed by any participants. If you are new to Python and Jypyter Notebooks, here are some reference that can help you:

* [Python Tutorial](https://www.python.org/about/gettingstarted/)
* [Jupyter Notebook documentation](https://jupyter.readthedocs.io/en/latest/index.html)
* [Jupyter Notebook tutorial](https://jupyter-notebook.readthedocs.io/en/stable/notebook.html)

# 2. Installing and importing python libraries
 [Go back to the "Table of contents"](#Table-of-contents)

<div class="alert alert-block alert-info">

<b> Tip: Importing modules </b>
* Python libraries need to be *imported* before they can be used;
* Imported libraries usually have a namespace;
* Portions of libraries, can be imported;
</div>

All librairies used in this notebook are already installed on the `conda` environment we just creadted. We hence simply need to import the requested librairies.

In [1]:
try:
    # to ignore warning message
    import warnings
    warnings.filterwarnings('ignore')
    from matplotlib import pyplot as plt # Visualization
    import cartopy.crs as ccrs # Projected visualizations
    import xarray as xr # Open, read and Process netCDF files
    import numpy as np # Data manupulation
    import cartopy # improved visualizations
    from glob import iglob # data access in file manager
    import glob   # data access in file manager
    from os.path import join # data access in file manager
    import pandas as pd # data manipulation
    import harp #Python interface of the atmospheric toolbox to open read and process Sentinel-5P products
    import imageio # create gif 
    import matplotlib.dates as mdates # date manipulation
    import matplotlib.patches as mpatches # to define patches (2D artist with a face color and an edge color such as rectangle, circles)
    from matplotlib.dates import DateFormatter # date manipulation
    import time #to manipulate time in Python and create timer
    import os
    
except ModuleNotFoundError:
    print ('Module import Error')
else:
    print('\nAll librairies properly loaded. Ready to start!')

Module import Error


# 3. Download Sentinel-5P data
 [Go back to the "Table of contents"](#Table-of-contents)

<p style='text-align: justify;'>
    In the earlier presentation, you learned how to download products one by one from the <a href="https://s5phub.copernicus.eu/dhus/"> Copernicus scihub </a> by manually looking for products matching your search criteria.
<p style='text-align: justify;'>
    The Copernicus Scihub also has an API to automatically retrieve data. This is useful when users need to download a large amount of products or download products on a regular basis (daily download of the products ingested in the last 24 hours for example). This section of the training introduces you to the Sentinel-5P data automatic retrieval process.

<div class="alert alert-block alert-info">
<b>Tip:</b> More information on the API hub can be found <a href="https://scihub.copernicus.eu/twiki/do/view/SciHubUserGuide/BatchScripting?redirectedfrom=SciHubUserGuide.8BatchScripting"> here </a>
</div>

<p style='text-align: justify;'>
In this example, we are going to download one NO2 product over Europe on March 24th 2019 using the API hub. To do so, we will use the dhusget.sh script which is an example script implemented for the API hub and freely available to any users of the Copernicus scihub.

<p style='text-align: justify;'>
First, you need to download the dhusget script <a href="https://scihub.copernicus.eu/twiki/pub/SciHubUserGuide/BatchScripting/dhusget.sh"> here</a>. We have downloaded it beforehand of this training session. You will find it in `/shared/S5P_NO2_092021/training_NO2_22092021/AuxData/bash/dhusget.sh`.

<p style='text-align: justify;'>
This script can be run via a simple command line. It searches for products based on several options that allow you to specify your search criteria. The options used in this training section are the following:
    
<b> Login option </b>
 - -d \<DHuS url\> : URL of the Data Hub Service to be polled - here `https://s5phub.copernicus.eu/dhus`
 - -u \<username\> : data hub username - here `'s5pguest'`
 - -p \<password\> : data hub password - here `'s5pguest'`
 
<b> Search query option </b>
 - -m \<mission name\> : sentinel mission name - here `'Sentinel-5'`
 - -S \<sensing_date_FROM\> : search for products with sensing date greater than the date and time specified by \<sensing_date_FROM\>. The date format is ISO 8601: YYYY-MM-DDThh:mm:ss.cccZ - here `'2019-03-24T00:00:00.000Z'`
 - -E \<sensing_date_TO\> : search for products with sensing date less than the date and time specified by \<sensing_date_TO>\. The date format is ISO 8601: YYYY-MM-DDThh:mm:ss.cccZ - here `'2019-03-24T20:00:00.000Z'`
 - -F \<free OpenSearch query\> : free text OpenSearch query. The query must be written enclosed by single apexes '<query>' (e.g. -F 'platformname:Sentinel-1 AND producttype:SLC' ). Note: the free text OpenSearch query can be combined with the other possible specified search options - here `'platformname:Sentinel-5 AND producttype:L2__NO2___ AND processinglevel:L2 AND processingmode:Offline'`
 - -c \<coordinates i.e.: lon_min,lat_max:lon_max,lat_min\> : coordinates of two opposite vertices of the rectangular area of interest - here `'8.3,42.1:8.2,42.0'`
    
<b> Search Result options </b>
 - -q \<XMLfile\> : write the OpenSearch query results in a specified XML file. Default file is './OSquery-result.xml' - here `/shared/S5P_NO2_092021/training_NO2_22092021/Processing/automatic_download/search_result_'date +%d%m%y\'_S5P.xml` (be aware, we must specify a relative path from the Notebook)
 - -C \<CSVfile\> : write the list of product results in a specified CSV file. Default file is './products-list.csv' - here `/shared/S5P_NO2_092021/training_NO2_22092021/Processing/automatic_download/products_list_`date +%d%m%y`_S5P.csv` (be aware, we must specify a relative path from the Notebook)
 - -l \<results\> : maximum number of results per page [1,2,3,4,..100]; default value = 25 - here `100`
    
<b> Download options </b>
 - -O \<path/filename\> : save the product netCDF file in the specified folder with the specified filename. - here `/shared/S5P_NO2_092021/training_NO2_22092021/Processing/automatic_download/ (the folder gets created automatically if it does not exist)
 - -o \<option\> : what to download - here `'product'`
 
 
 Within the script `dhusget.sh` and the [website](https://scihub.copernicus.eu/twiki/do/view/SciHubUserGuide/BatchScripting#dhusget_script) (where the bash can be downloaded) there is a description of all available options.

<div class="alert alert-block alert-info">
    
<b>Tip:</b> When calling a linux command from a notebook, insert a `!` at the beginning of the command line. To split a long command line into several smaller ones, use `\` at the end of each line.
    
</div>

In [ ]:
# Command line to download data
!/shared/S5P_NO2_092021/training_NO2_22092021/AuxData/bash/dhusget.sh \
-d 'https://s5phub.copernicus.eu/dhus' \
-u 's5pguest' -p 's5pguest' \
-m 'Sentinel-5' \
-S 2019-03-24T00:00:00.000Z -E 2019-03-24T20:00:00.000Z \
-F 'platformname:Sentinel-5 AND producttype:L2__NO2___ AND processinglevel:L2 AND processingmode:Offline' \
-l 100 \
-c 8.3,42.1:8.2,42.0 \
-q ../Processing/automatic_download/search_result_`date +%d%m%y`_S5P.xml \
-C ../Processing/automatic_download/products_list_`date +%d%m%y`_S5P.csv \
-O '/shared/S5P_NO2_092021/training_NO2_22092021/Processing/automatic_download/' \
-o 'product'

For information, we have chosen a tiny delimiting box (8.3,42.1:8.2,42.0) to retrieve only one product during the session and spare some time downloading the products.

<div class="alert alert-block alert-warning">
    
The above cell is an example of how to automatically download S5P products. To spare time, we will use products that are already downloaded in the following parts of the notebook. If you want to dowload them later or download other products of your choice, you should modify the command line above! We will see a second example in [part 5.1](#5.1.-Concatenation)
    
</div>

# 4. Explore the netCDF file content
 [Go back to the "Table of contents"](#Table-of-contents)
 
<img style="float: left;" src=graphics/S5p_FileStructre.PNG width='400'/>

Before starting, it is important to remind how the Sentinel-5P netCDF files are structured (see above image). There are different groups used to organise the data and make it easier to find what you are looking for.

There are two groups in each Sentinel-5P netCDF file: PRODUCT and METADATA which themselves contain other subgroups.
* **PRODUCT**: This group stores the main fields (latitude, longitude, main variables). The `qa_value` parameter summarizes the processing flags into a continuous value, giving a quality percentage: 100% is the most optimal value, 0% is a failure, in between lies a continuum of values;
* **METADATA**: This group collects metadata items. These metadata standards are all meant to facilitate dataset discovery. The metadata will be stored as attributes, while grouping attributes that belong to a specific standard will be done by using sub-groups in the Metadata group.

In the following parts of this Notebook, we will use the <a href="http://xarray.pydata.org/en/stable/"> `xarray` </a> Python library to open and manipulate the products. We will also use the <a href="https://atmospherictoolbox.org/"> Atmospheric Toolbox </a> and its <a href="https://atmospherictoolbox.org/harp/"> HARP </a> component to preprocess the files. 

One important thing to note is that `xarray`  doesn't handle NetCDF groups and groups must be imported one by one. So if you want to interactively discover all groups of a product and the variables they contain, we advise you to use `Panoply` as shown in the previous exercise (the application is installled in your virtual machine).

## 4.1. Product Exploration

In this part, we are going to work with only one file to get familiar with xarray and python commands.

Let's check the content of the Sentinel-5P NO2 file we just downloaded. We first need to define:
* `RootPath`: the path pointing to the data directory
* `FName`: the path of the NetCDF file (`Rootpath`+filename.nc) 

These variables can of course be changed later on depending on the netCDF filename and where the data is stored. 

In [ ]:
# Define your data directory Rootpath and your filename FName
RootPath = '/shared/S5P_NO2_092021/training_NO2_22092021/Processing/automatic_download/'
FName = RootPath + 'S5P_OFFL_L2__NO2____20190324T111531_20190324T125701_07478_01_010300_20190330T125634.nc'

Now that we have told Jupyter which data we are going to work with and where it is, we need to open this data.
The following cell opens the netCDF file (`xr.open_dataset()` function) as an <a href="http://xarray.pydata.org/en/stable/data-structures.html#dataset"> xarray.Dataset</a> and allows you to interactively browse the content of the file. 

<div class="alert alert-block alert-info">
    
<b>Tip:</b> To access the <strong>PRODUCT</strong> and <strong>METADATA</strong> group, the keyword `group` has to be passed as argument to the open_dataset() function as `xarray` does not support netCDF groups as part of the DataSet data model.
</div>

In [ ]:
# Open product
xr.open_dataset(FName,group='PRODUCT')  # Ingest group 'PRODUCT' group into an xarray.Dataset

Now, to store the product content into an xarray.Dataset called `FIn` and display the content of the dataset, you need to execute the following cell:

In [ ]:
####  Open the netCDF file with xr.open_dataset() and get general information ####

## File
FIn = xr.open_dataset(FName,group='PRODUCT')   #Handling of the netCDf file

# Show Header: global attributes
FIn

In [ ]:
### Print the different variables of a netCDF file ###
FIn.data_vars

This function displays all available variables in the `product` group of a file along with the dimensions they depend on and their data type (`float32` here, meaning that each value is a float coded on 32 bits). You can see that the present variables in the files depend on several coordinates. To get information about these coordinates:

In [ ]:
###  Print the coordinates of a netCDF file ###
FIn.coords

If you want to store the values of a variable in an array, type `MyArray = FIn.variable`. For example for the time variable:


In [ ]:
### Store only the values in a variable ###
VarTime=FIn.time.values

In [ ]:
print(VarTime)

To convert a date from a format to another, use the `astype()` function

In [ ]:
#astype allows to choose the precision of the date: D for Day, Y for years, M for months, h for hours etc

print(VarTime.astype('datetime64[D]'))

## 4.2. Visualisation of one NO2 product

In this part, we will map the product we just opened with `xarray`. Each Sentinel-5P product corresponds to one satellite track (a pole to pole half orbit). We have previously seen the exploration and manipulation of the different groups and variables in these groups. Now we would like to access the actual NO<sub>2</sub> measurement in order to plot it. This quantity is stored in the `nitrogen_tropospheric_column` variable.

<div class="alert alert-block alert-info">
    
<b>Tip:</b> To access a variable `Var` in a xarray.Dataset `FIn`, use `Myvar=FIn[Var]`. This will store the variable as an <a href="http://xarray.pydata.org/en/stable/data-structures.html#dataarray"> `xarray.DataArray` </a>
    
</div>

In [ ]:
### Get the time variable ###
date=VarTime.astype('datetime64[D]')

###  Store the variable NO2
VarNO2=FIn['nitrogendioxide_tropospheric_column'] #store the NO2 measurement variable of FIn in VarNO2

FIn.close() #Close the xarray Dataset as we got all the needed variables

In [ ]:
# Display the characteristics of VarNO2
print(VarNO2)

Xarray provides convenient plot tools for mapping DataArray. In this notebook, we will extensively use the <a href="http://xarray.pydata.org/en/stable/generated/xarray.plot.pcolormesh.html"> xarray.plot.pcolormesh() </a> function which simply wraps the <a href="https://matplotlib.org/api/_as_gen/matplotlib.pyplot.pcolormesh.html#matplotlib.pyplot.pcolormesh"> matplotlib.pyplot.pcolormesh() </a> function. To improve the map of the nitrogen dioxyde column in the product, we will also use the Plate Carree projection from <a href="https://scitools.org.uk/cartopy/docs/latest/"> cartopy </a>. `cartopy` is a python package designed for geospatial data processing in order to produce maps and other geospatial data analysis.

In [ ]:
# Visualization using Cartopy and Plate Carree projection
fig=plt.figure(figsize=(40,15)) # create a figure frame and set up the figure size
ax = plt.axes(projection=ccrs.PlateCarree()) # Creates an empty subplot with Plate Carree projection

# Plot VarNO2 where we remove the time dimension `VarNO2[0]`
im=VarNO2[0].plot.pcolormesh(ax=ax, x='longitude', y='latitude', 
                             add_colorbar=False, cmap='jet', \
                           transform=ccrs.PlateCarree(),vmin=0,vmax=0.0001) 

# Define the figure colorbar
cbar=fig.colorbar(im, ax=ax, pad=0.01, format='%.1e')
cbar.ax.tick_params(labelsize=16)
cbar.set_label(label='Tropospheric vertical column of NO2 (mol.m-2)',fontsize=16)

#ax.add_feature(cartopy.feature.RIVERS) # add river
ax.set_title('S-5p L2 NO$_2$ ({}) '.format(str(date[0])), fontsize=50) # add title
ax.coastlines('10m')  # add coastline
ax.stock_img() # add earth background
ax.gridlines()  # add grid line

# Save figure to file
plt.savefig('/shared/S5P_NO2_092021/training_NO2_22092021/Processing/figures/NO2_one_track.png', facecolor='white', bbox_inches='tight', dpi=300)

We just mapped one raw Sentinel-5P NO2 product using xarray and cartopy, without applying any kind of preprocessing. Different indicators (like `qa_value` introduced above) come with the data and describe the context around the measurement. Moreover, mapping only one product at a time offers limited applications. Users are more interested in getting all available samples over their region of interest for a given period of time in order to identify trends or punctual phenomena. Studying NO2 concentration over a given area with satellite data usually involves computing 10-15 day averages of the NO2 column measured in order to iron out the effects of weather.

The next part describes how to spatially and temporaly filter and reproject Sentinel-5P products, how to keep only the best quality pixels and how to merge several single track products into one product. These exercises will rely on the Atmospheric toolbox and more particularly on its HARP component which was introduced to you earlier.

# 5. Visualisation of NO2 for a day
 [Go back to the "Table of contents"](#Table-of-contents)

NO2 is an important trace gas in the Earth's atmosphere. It is present in both the troposphere and the stratosphere as a result of:
   - Anthropogenic activities (traffic, industrial fossil fuel combustion, biomass burning)
   - Natural processes (wildfires, lightning)

NO2 has a short lifetime in the atmosphere (from a few hours up to a half day) and remains relatively close to its source. This is why it is often used to as a proxy to follow the evolution of Anthropogenic activies. You will find more information in the <a href="https://sentinel.esa.int/documents/247904/2476257/Sentinel-5P-TROPOMI-ATBD-NO2-data-products"> TROPOMI ATBD of the tropospheric and total NO2 data products </a>

In this part we analyze NO2 concentration in the world troposphere on march 24th 2019. 

## 5.1. Concatenation

All files have been downloaded ahead of this training session for this exercise. The command line used to download them is displayed below:
* -E option has been updated
* no need of the -c option as we downloaded all products acquired over the course of March 24th 2019.


In [ ]:
# Command line to download data, uncomment the -o option if you want to download the product
!/shared/S5P_NO2_092021/training_NO2_22092021/AuxData/bash/dhusget.sh \
-d 'https://s5phub.copernicus.eu/dhus' \
-u 's5pguest' -p 's5pguest' \
-m 'Sentinel-5' \
-S 2019-03-24T00:00:00.000Z -E 2019-03-25T02:0:00.000Z \
-F 'platformname:Sentinel-5 AND producttype:L2__NO2___ AND processinglevel:L2 AND processingmode:Offline' \
-l 100 \
-q ../Processing/automatic_download/search_result_`date +%d%m%y`_S5P.xml \
-C ../Processing/automatic_download/products_list_`date +%d%m%y`_S5P.csv \
-O '/shared/S5P_NO2_092021/training_NO2_22092021/Processing/automatic_download/' \
#-o 'product'

<div class="alert alert-block alert-info">
    
<b>Tip:</b> The Earth is usually covered by 14 to 15 S5P products in a single day. You may need to increase the end date by a few hours in order to find all products for a day.
    
</div>

To highlight the various possibilities of the Atmospheric Toolbox, this exercise presents three different and equivalent ways of merging Sentinel-5P L2 products into a gridded L3 averaged product. For each different method, we will calculate the computing time, to determine the fastest way of merging NO2 products.

The characteristics of the averaged NO2 products are the following:
* Destination grid:
    * Lon: [-180°E,180°E] 3600 bins with a step of 0.1°;
    * Lat: [-90°N,90°N] 1800 bins with a step of 0.1°;
* We will filter out the observations with a quality flag less than 75. This is the recommended threshold value in the <a href="https://sentinel.esa.int/documents/247904/2474726/Sentinel-5P-Level-2-Product-User-Manual-Nitrogen-Dioxide"> Sentinel-5P NO2 Product User Manual </a>. This quality indicator depends on many factor including cloud cover, surface albedo, presence of snow-ice, saturation, geometry etc. These aspects are taken into account in the definition of the "quality assurance value" (qa_value), available for each individual observation, which provides the users with an easy filter to remove less accurate observations. The qa_value is a continuous variable, ranging from 0 (error) to 1 (optimal retrieval). The main flag for data usage is as follows: 
    * qa_value > 0.75: This is the recommended pixel filter. It removes cloud-covered scenes (cloud radiance fraction> 0.5), partially snow/ice covered scenes, errors, and problematic retrievals.
    * qa_value > 0.50: Compared to the stricter filter, this adds the good quality retrievals over clouds and over scenes covered by snow/ice. Errors and problematic retrievals are still filtered out. In particular, this filter may be useful for assimilation and model comparison studies.



### 5.1.1. Harpmerge in command line

The first method will consist in using the **harpmerge** command line tool. As we saw in the presentation, harpmerge combines multiple products from files or directory by appending them accross the time dimension and storing them into a single output file. For this we will use the command line below:

Harpmerge <font color='red'>–ap ‘operations_list’</font> <font color='green'>–a ‘operations_list’ </font> <font color='blue'>–o ‘ingestion_option’ </font> <font color='black'>input_files</font> <font color='brown'> output_Directory/output_file</font>

`keep()` is an operation used in this example, which specifies that all variables marked for inclusion will be kept in the ingested product, while all other variables will be excluded.

To assess the processing time, we will also use the `time` method of python. This method returns the time as a floating point number expressed in seconds since the epoch, in UTC.

<div class="alert alert-block alert-info">
    
<b>Tip:</b> When calling a linux command from a notebook, insert a `!` at the beginning of the command line. To split a long command line into several smaller ones, use `\` at the end of each line.
    
</div>

In [ ]:
#Starting time of the cell
t0= time.time()

#harpmerge in command line
!harpmerge -ap 'bin(); squash(time, (latitude,longitude,tropospheric_NO2_column_number_density))' \
-a 'tropospheric_NO2_column_number_density_validity>75; \
bin_spatial(1800,-90,0.1,3600,-180,0.1); \
derive(longitude {longitude});derive(latitude {latitude}); \
keep(latitude,longitude,tropospheric_NO2_column_number_density,weight)' \
/shared/S5P_NO2_092021/training_NO2_22092021/Original/NO2_24032019/S5P_OFFL_L2__NO2____20190*.nc \
/shared/S5P_NO2_092021/training_NO2_22092021/Processing/merged_files/command_line/converted_NO2_command_line.nc

#End time for command line
t1 = time.time()

command_line_time = t1-t0

print("Command line time: {} seconds".format(command_line_time))

<div class="alert alert-block alert-info">
    
<b>Tip:</b> Remember that when ingesting products, HARP will rename the product variables. The HARP variable mapping for S5P is available on <a href="https://stcorp.github.io/harp/doc/html/ingestions/index.html#sentinel-5p-products"> this webpage </a>. 

For example, the `nitrogen_tropospheric_column` variable will be labelled as `tropospheric_NO2_column_number_density` in the corresponding HARP product. 
        
In the same way, `qa_value` will be labelled as `nitrogen_tropospheric_column_number_density_validity` and rescaled between 0 and 100 (rather than 0-1) in the corresponding HARP products
    
</div>

### 5.1.2. Harp.import_product() without preprocessing

The second method will consist in using the <a href="https://stcorp.github.io/harp/doc/html/python.html"> python interface of HARP </a> and the `harp.import_product()` function. We have already imported the `harp` Python library in [part 2](#2.-Importing-python-libraries).   This function works as the `harpmerge` command line tool. It takes a single input file or a set of input files. The operations to be applied to each L2 product before they are merged are specified through the `operations=` (equivalent to the `-a` of `harpmerge`) argument (should be specified as a semi-colon separated string of operations). The operations to be applied to the merged product are specified via the `post_operations=` (equivalent to the `-ap` of `harpmerge`) argument.

`harp.import_product()` returns the imported or the merged product (depending on whether the input is a single file or a set of files). If you want to export the output to a NetCDF file, use the `harp.export_product()` function.

In [ ]:
# Specify input and output of harp.import_product
input_files='/shared/S5P_NO2_092021/training_NO2_22092021/Original/NO2_24032019/S5P_OFFL_L2__NO2____20190*.nc' #All S5P L2 NO2 product on 24/03/2019
export_path='/shared/S5P_NO2_092021/training_NO2_22092021/Processing/merged_files/without_preprocessing/converted_NO2_without_preprocessing.nc' #NetCDF file where merged product will be stored

In [ ]:
#Starting time of the cell
t0= time.time()

#Use harp.import_product python function to generate 
Converted_NO2 = harp.import_product(input_files, \
                      operations= "tropospheric_NO2_column_number_density_validity>75; \
                      bin_spatial(1800,-90,0.1,3600,-180,0.1); \
                      derive(latitude {latitude}); derive(longitude {longitude}); \
                      keep(latitude,longitude,tropospheric_NO2_column_number_density,weight)", \
                      post_operations="bin(); squash(time, (latitude,longitude,tropospheric_NO2_column_number_density))"                   
                      )
        
harp.export_product(Converted_NO2, export_path,file_format="netcdf")

#End time for merged product
t1 = time.time()

without_preprocessing_time = t1-t0

print("time without preprocessing: {} seconds".format(without_preprocessing_time))

### 5.1.3. Harp.import_product() with preprocessing


As shown by the 2 previous cells, generating L3 products from L2 products can take up a lot of computing time when working with many files. When exploring Sentinel-5P data, we noticed that they contain many variables and only a few of them are of interest for our study. This is why pre-processing the files before merging them can greatly optimize the performance. To this end, we use the `harp.import_product()` function on each S5P L2 NO<sub>2</sub> file to remove the variables we are not interested in and crop the full orbit files to our area of interest. This way, we will work with much lighter L2 products. 

In the code cell below, we first define the `input_path` and `export_path`. Then we collect in a single variable all input file paths. For this, we use the following functions:
   - `join()`: allows to gather different parts of a path, here the path of the folder and the name of the files
   - `glob.glob()`: returns a potentially empty list of paths matching the pattern pathname
   - `listdir()`: returns a list containing the names of the entries in the directory given by path in arbitrary order
   - `sorted()`: returns a sorted list of the specified iterable object

We ill use the `list_files` variable to store each pre-processed file with the same name as its corresponding input file.

***
<div class="alert alert-block alert-warning">
<b>WARNING</b>
    
***  

In the following cell we will use a for loop. It is important to keep in mind that the Python language starts at index 0 (0-based). For example, `files_input[0:3]` will return the values of the `files_input` variable from index 0 to 2. Python does not take into account the last index.

In [ ]:
input_path= '/shared/S5P_NO2_092021/training_NO2_22092021/Original/NO2_24032019/' #Directory of the original S5P files
export_path='/shared/S5P_NO2_092021/training_NO2_22092021/Processing/pre_processed_files/NO2_24032019' #output directory

# the function `list` recovers the name of all files in the input_path folder
list_files = sorted(os.listdir(input_path))

# Make sure that list_files contains only S5P files (if the input directory contains other files)
for file in list_files:
    if file.startswith("S5P_OFFL_")==False:
        list_files.remove(file)

# Get the name of all files in the folder and sort them in alphabetic order
files_input= sorted(glob.glob(join(input_path, 'S5P_OFFL_*.nc')))

#Starting time of the preprocessing
t0 = time.time()

#Pre-processing loop on each file in input_path to get rid of unnecessary variables (keep()) and crop to our ROI
for i in range(len(files_input)):
    #Pre-processing
    Converted_NO2=harp.import_product(files_input[i], \
            operations= "keep(latitude,latitude_bounds,longitude,longitude_bounds, \
            tropospheric_NO2_column_number_density,tropospheric_NO2_column_number_density_validity,datetime_start)")
    #Export of the preprocessed file to export_path
    harp.export_product(Converted_NO2, join(export_path, list_files[i]),file_format="netcdf") 
    print("product", os.path.basename(files_input[i]),"pre-processed")

#End time of the pre-precessing
t1 = time.time()

pre_processing_time = t1-t0

print("Pre-processing time: {} seconds".format(pre_processing_time))

Now let's merge together the pre-processed files.

In [ ]:
# Specify input and output of harp.import_product
input_files='/shared/S5P_NO2_092021/training_NO2_22092021/Processing/pre_processed_files/NO2_24032019/S5P_OFFL_L2__NO2_*.nc' #All S5P preprocessed product
export_file='/shared/S5P_NO2_092021/training_NO2_22092021/Processing/merged_files/with_preprocessing/converted_NO2_with_preprocessing.nc' #outputfile

#Use harp.import_product python function to generate the merged product

#Start time of the processing
t0 = time.time()

#Merging of the pre-processed files
Converted_NO2 = harp.import_product(input_files, \
                      operations= "tropospheric_NO2_column_number_density_validity>75; \
                      bin_spatial(1800,-90,0.1,3600,-180,0.1); \
                      derive(latitude {latitude}); derive(longitude {longitude}); \
                      keep(latitude,longitude,tropospheric_NO2_column_number_density,weight)", \
                      post_operations="bin(); squash(time, (latitude,longitude,tropospheric_NO2_column_number_density))"                   
                      )

#Export of the merged file        
harp.export_product(Converted_NO2, export_file,file_format="netcdf")

#End time of the processing
t1 = time.time()

processing_time = t1-t0

#Total processing time accounts for the preprocessing time and the actual merging
total_processing_time=pre_processing_time+processing_time

print("Processing time: {} seconds".format(processing_time))
print("Total processing time with preprocessing: {} seconds".format(total_processing_time))



As you can see, the computing time is much shorter when pre-processing the NO2 files beforehand. Remember, we have preprocessed the original files by reducing the number of variables.

It is also possible to restrict the geographical extent to a square above your region of interest 



<div class="alert alert-block alert-info">
<b>Tip:</b> When running HARP, different Exceptions can take place. (click <a href='https://stcorp.github.io/harp/doc/html/python.html#examples'> here </a> for more details on HARP Exceptions). One of the most common you may face is the <i>NoDataError</i>, which is raised when the product returned from an import contains no variables, or variables without data. This can especially happen when importing single products. This can be caused for example by a product that does not overlap with your AOI or because all ground pixels have been filtered out by the <i>qa_value</i>. 
 
This kind of Exception can be caught with `try:` and `except NoDataError:` so that the processing keeps working if a `NoDataError` exception is raised
</div>

## 5.2. Visualisation of NO2 

Now that we have created the merged products on 24th march 2019 using 3 differents approaches, we can map the averaged NO2 column and check that the merged products are the same.

Before opening the merged products we `def`ine a python function named `figures()`. This way we will be able to generate figures by simply calling the `figure` function rather than repeating the same code.

This function takes as input the following variables:
* data: the data that we want to visualize (must be an `xr.DataArray`)
* ax: the axe used for the figure (a `plt.subplot` with a defined projection)
* title: title of the figure (string)
* shrink: the size of the colorbar between 0 and 1 (float)

We will use the `pcolormesh()` function for the colour mapping.




In [ ]:
def figures(data:xr.DataArray,ax:plt.subplot,title:str, shrink:float, alpha:float) -> plt.subplot:
    """
    Creates a coloured map of NO2 tropospheric column. 
    Args:
        (xr.dataArray) data, NO2 tropospheric column
        (subplot) ax, supblot with a defined projection
        (str) title, title of your figure
        (float) shrink, size of color bar
        (float) alpha, The alpha blending value, between 0 (transparent) and 1 (opaque)
    Returns:
        (subplot) coloured map of the NO2 tropospheric column
    """
        
    
    # Coloured Map of O3 total column levels
    im1=data.plot.pcolormesh(ax=ax, x='longitude', y='latitude',add_colorbar=False, 
                            cmap='jet', transform=ccrs.PlateCarree(), vmin=0, vmax=0.12, alpha=alpha)
    
    # Figure settings
    ax.set_title(title ,fontsize=35)
    ax.add_feature(cartopy.feature.RIVERS)
    ax.coastlines('10m',linewidth=2)
    ax.stock_img() # add earth background
    ax.gridlines()
    cbar = plt.colorbar(im1,ax=ax, orientation="vertical",pad=0,shrink=shrink)#pad adjust the space between plots and colorbar
    cbar.ax.tick_params(labelsize=20)
    
    return (ax)


### 5.2.1. Visualisation of the three outputs

First, we will open each merged product with the cell below:


In [ ]:
# Directory where the data is stored
FName_command_line='/shared/S5P_NO2_092021/training_NO2_22092021/Processing/merged_files/command_line/converted_NO2_command_line.nc'
FName_without_preprocessing='/shared/S5P_NO2_092021/training_NO2_22092021/Processing/merged_files/without_preprocessing/converted_NO2_without_preprocessing.nc'
FName_with_preprocessing='/shared/S5P_NO2_092021/training_NO2_22092021/Processing/merged_files/with_preprocessing/converted_NO2_with_preprocessing.nc'

# Put the data in the Datasets
FIn_cl = xr.open_dataset(FName_command_line)
FIn_noprep = xr.open_dataset(FName_without_preprocessing)
FIn_prep = xr.open_dataset(FName_with_preprocessing)
# Put the tropospheric NO2 data in a variable
# We mutiply by 1000 to convert from [mol.m-2] to [mmol.m-2] 
VarNO2_cl=FIn_cl['tropospheric_NO2_column_number_density']*(10**3)
VarNO2_noprep=FIn_noprep['tropospheric_NO2_column_number_density']*(10**3)
VarNO2_prep=FIn_prep['tropospheric_NO2_column_number_density']*(10**3)

The following cell creates one figure with three subplots on the same column (one subplot per approach)

In [ ]:
#Ax definition
fig,axes=plt.subplots(3,1,figsize=(30,50)) # create a figure frame with three lines and one column and set up the figure size
axes=axes.ravel() #Turn the axes into a 1D array in order to loop on the different axes
titles=['With command line','Without preprocessing','With preprocessing'] #Array of the titles
variables=[VarNO2_cl,VarNO2_noprep,VarNO2_prep] #Array of the variables to be plotted
shrink=0.8
alpha=0.8

for i in range(0,len(axes)):
    j=311+i #to define the ax position in the subplot 311, 312 and 313
    axes[i]=plt.subplot(j,projection=ccrs.PlateCarree()) # Creates an empty subplot with Orthographic projection

    #Title definition
    title =titles[i] # title of the current ax

    #Function call
    figures(variables[i],axes[i],title,shrink,alpha) #figures function call on the current ax for the current variable

#Define the general title
fig.suptitle('S-5p averaged NO$_2$ tropospheric column on 24/03/2019(mmol.m-2)',fontsize=35,y=0.88)
#Adjust the vertical space between subplots
fig.subplots_adjust(hspace=0.01)
# Save figure to file
plt.savefig('/shared/S5P_NO2_092021/training_NO2_22092021/Processing/figures/NO2_three_methods', facecolor='white',\
            bbox_inches='tight', dpi=300)

The image above shows three identicals figures. This indicates that the three approaches processed the input products in an equivalent way.

We notice some data gaps on our maps. Indeed, in the previous section, when generrating the daily merged products, we discarded the lowest quality pixels. Hence, harp did not take into account the pixels with a `qa_value<0.75` (or a `tropospheric_NO2_column_number_density_validity<75` in harp convention). These pixels were most probably covered in clouds or were suffering from a data retrieval issue.  This is why it is advised to study the NO2 averaged tropospheric column over a 2 week period. This ensures the production of gap-free maps and this irons out the effect of weather on the NO2 tropospheric column. 
Indeed, we are interested in studying the anthropogenic emissions of NO2. Focusing on only one day at a time could be misleading as the tropospheric NO2 can be dispersed by winds or our area of interest covered in clouds when the measurements were collected.

As a further assessment we can directly chech the equality of the three generated products thanks to the `xarray.dataArray.equals()` function. This function will return `True` if two DataArrays have the same dimensions, coordinates and values; `False` otherwise.

In [ ]:
equal1=VarNO2_cl.equals(VarNO2_noprep)
equal2=VarNO2_noprep.equals(VarNO2_prep)
if (equal1 and equal2):
    print("The three products are equivalent!")
else:
    print("The three products are not equivalent...")

### 5.2.2. Visualisation of NO2 over Europe

In this part, we are going to focus over Europe. For this, we will crop the product to an area of interest defined by a latitude range [31;58]°N and a longitude range [-34;34]°E. We use the `sel()` function (more information [here](http://xarray.pydata.org/en/stable/generated/xarray.Dataset.sel.html)). The function returns a new dataset with each array indexed by tick labels along the specified dimension(s), here the `longitude` & `latitude` .

In [ ]:
#Deine lat and lon extent
lon_min=-34
lon_max=34
lat_min=32
lat_max=58

In [ ]:
NO2_Europe=VarNO2_prep.sel(latitude=slice(lat_min,lat_max),longitude=slice(lon_min,lon_max))

The following cell creates one figure with two subplots. The first image (left subplot) is the tropospheric NO2 on 24/03/2019 over the world with our area of interest highlighted by the red rectangle. The second image (right subplot) is the tropospheric NO2 on 24/03/2019 over Europe.

In [ ]:
shrink=0.4
alpha=0.6

fig,(axe1,axe2)=plt.subplots(1,2,figsize=(30,15))

axe1 = plt.subplot(121, projection=ccrs.PlateCarree())
title = 'World'
figures(VarNO2_prep,axe1,title,shrink,alpha)
# add the red rectangle, that corresponds to the area of the zoom on Europe
axe1.add_patch(mpatches.Rectangle(xy=[lon_min, lat_min], width=lon_max-lon_min, height=lat_max-lat_min,
                                    edgecolor='red', facecolor='None', linewidth=4,
                                    transform=ccrs.PlateCarree()))

axe2 = plt.subplot(122, projection=ccrs.PlateCarree())
title = 'zoom over Europe'
figures(NO2_Europe,axe2,title,shrink,alpha)
axe2.set_extent([lon_min, lon_max, lat_min, lat_max], crs=ccrs.PlateCarree())

#Define the general title
fig.suptitle('Tropospheric NO2 column on 24/03/2019 (mmol.m-2)',fontsize=35,y=0.73)

# Save figure to file
plt.savefig('/shared/S5P_NO2_092021/training_NO2_22092021/Processing/figures/NO2_24032019_zoom_in_Europe', facecolor='white'
            \ bbox_inches='tight', dpi=300)

Even though the data gaps are still present, some areas show higher concentrations of NO2:
* Barcelona,Spain
* Near Marseille,France
* The region of Lombardy in Italy
* The region of Ruhr in Germany
* Londonian area, United Kingdom

In the second part of this training, we will compare the tropospheric column of NO2 over Europe between 2019 and 2020 for a defined period. Indeed, starting mid March 2020, there were quarantines or similar restrictions (variously described as stay-at-home orders, shelter-in-place orders, shutdowns or lockdowns) happening in many countries and territories around the world, related to the COVID-19 pandemic and established to prevent the further spread of the virus. In particular, Europe's population was under lockdown, having been asked or ordered to stay at home by their governments.

During this lockdown, air quality over most European cities improved significantly due to the dramatic drop of traffic and industrial activities.

# 6. Conclusion
 [Go back to the "Table of contents"](#Table-of-contents)

<div class="alert alert-block alert-success">
    <b>CONGRATULATIONS</b><br>
  
--- 

#### And thank you for your attention! :) We hope you enjoyed this training on Sentinel 5P provided by RUS - Copernicus for free, thanks to ESA and the European Commission.

#### Now let's try to download new data and variables and to access and visualize them. You can try to make new maps and plots. And don't forget to try the other [Sentinel 5P products](https://s5phub.copernicus.eu/dhus/)!

This training course is over but we'd love to hear from you about how we could improve it (topics, tools, storytelling, format, speed etc). 